In [13]:
import os
import cv2
import numpy as np
import csv
from skimage.feature import graycomatrix, graycoprops


In [14]:
def calculate_glcm_features(image, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):



    # Compute GLCM
    glcm = graycomatrix(image, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)



    # Calculate GLCM properties
    contrast = graycoprops(glcm, 'contrast').flatten()[0]
    homogeneity = graycoprops(glcm, 'homogeneity').flatten()[0]
    energy = graycoprops(glcm, 'energy').flatten()[0]
    correlation = graycoprops(glcm, 'correlation').flatten()[0]

    # Calculate entropy
    glcm_probabilities = glcm / np.sum(glcm)
    entropy = -np.sum(glcm_probabilities * np.log2(glcm_probabilities + (glcm_probabilities == 0)))

    return {
        'contrast': contrast,
        'homogeneity': homogeneity,
        'energy': energy,
        'correlation': correlation,
        'entropy': entropy
    }


In [15]:
def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Extract the blue channel
    blue_channel = image[:, :, 0]

    # Apply thresholding
    _, thresholded = cv2.threshold(blue_channel, 90, 255, cv2.THRESH_BINARY_INV)

    return thresholded

In [16]:
def process_images_in_folder(folder_path, output_csv):
    results = []

    label = 0 if "Negative_" in os.path.basename(folder_path) else 1
    
    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            file_path = os.path.join(folder_path, filename)

            # Preprocess the image
            preprocessed_image = preprocess_image(file_path)

            # Calculate GLCM features
            glcm_features = calculate_glcm_features(preprocessed_image)

            # Append the results
            results.append({
                'filename': filename,
                'contrast': glcm_features['contrast'],
                'homogeneity': glcm_features['homogeneity'],
                'energy': glcm_features['energy'],
                'correlation': glcm_features['correlation'],
                'entropy': glcm_features['entropy'],
                'label': label
            })
                # Write the results to a CSV file
    with open(output_csv, mode='w', newline='') as file:
        fieldnames = ['filename', 'contrast', 'homogeneity', 'energy', 'correlation', 'entropy', 'label']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)

In [17]:
def process_multiple_folders(base_folder):
    # Iterate over all subfolders in the base folder
    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        if os.path.isdir(folder_path):
            output_csv = f'GLCM/{folder_name}_texture_features.csv'
            process_images_in_folder(folder_path, output_csv)
            print(f"GLCM features have been written to {output_csv}")

In [18]:
if __name__ == "__main__":
    folder = ['Rice_photos/BC-15', 'Rice_photos/Huongthom', 'Rice_photos/Nep87', 'Rice_photos/Q5', 'Rice_photos/Thien_uu', 'Rice_photos/Xi23']
    for i in range(len(folder)):
        base_folder = folder[i]
        process_multiple_folders(base_folder)


GLCM features have been written to GLCM/BC-15_texture_features.csv
GLCM features have been written to GLCM/Negative_BC-15_texture_features.csv
GLCM features have been written to GLCM/Huong_thom-1_texture_features.csv
GLCM features have been written to GLCM/Negative_Huong_thom-1_texture_features.csv
GLCM features have been written to GLCM/Negative_Nep-87_texture_features.csv
GLCM features have been written to GLCM/Nep-87_texture_features.csv
GLCM features have been written to GLCM/Negative_Q-5_modify_texture_features.csv
GLCM features have been written to GLCM/Q-5_modify_texture_features.csv
GLCM features have been written to GLCM/Negative_Thien_uu-8_texture_features.csv
GLCM features have been written to GLCM/Thien_uu-8_texture_features.csv
GLCM features have been written to GLCM/Negative_Xi-23_texture_features.csv
GLCM features have been written to GLCM/Xi-23_texture_features.csv
